In [1]:
import numpy as np
import pandas as pd

def r_Q_continuous(
    sales_order_file_path,
    initial_inventory=10000,
    holding_cost=1,
    ordering_cost=50,
    desired_service_level=0.95,
    max_shelf_space_for_SKU=124400,
    MOQ=1000,
    today_date='2021-04-01',
    lead_time=5,
    back_orders=50,
    open_orders=100,
    review_period=3,
    simulation_days=365,
    num_simulations=1000
):
    # Read the sales order data
    demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)

    # Rename the column if necessary
    if 'total qty' in demand_data.columns:
        demand_data.rename(columns={'total qty': 'total_qty'}, inplace=True)

    # Convert 'total_qty' to numeric and fill missing values
    demand_data['total_qty'] = pd.to_numeric(demand_data['total_qty'], errors='coerce').fillna(0)

    # Convert today's date to datetime
    today_date = pd.to_datetime(today_date)

    # Standard deviation of demand over lead time
    def standard_dev_over_lead_time(demand_data, today_date, lead_time):
        todays_index = demand_data.index.get_loc(today_date)
        end_index = todays_index + lead_time
        lead_time_df = demand_data.iloc[todays_index:end_index]
        return lead_time_df['total_qty'].std()

    demand_std_dev = standard_dev_over_lead_time(demand_data, today_date, lead_time)

    # Forecasted demand over lead time
    def forecasted_demand_over_lead_time(demand_data, today_date, lead_time):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index + 1
        end_index = todays_index + lead_time
        lead_time_df = demand_data.iloc[start_index:end_index]
        return lead_time_df['total_qty'].sum()

    dd_over_lead_time = forecasted_demand_over_lead_time(demand_data, today_date, lead_time)

    # Function to calculate safety stock
    def safety_stock_calculation(demand_std_dev, lead_time, Z):
        Z_score = abs(np.percentile(np.random.normal(size=1000000), 100 * Z))
        return Z_score * (lead_time ** 0.5) * demand_std_dev

    safety_stock = safety_stock_calculation(demand_std_dev, lead_time, desired_service_level)

    # Forecasted demand till next review period
    def forecasted_demand_till_next_review(demand_data, today_date, review_period):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index
        end_index = todays_index + review_period
        review_period_df = demand_data.iloc[start_index:end_index]
        return review_period_df['total_qty'].sum()

    dd_till_next_review = forecasted_demand_till_next_review(demand_data, today_date, review_period)

    # Dynamic reorder point calculation
    def reorder_point_calculation(safety_stock, dd_over_lead_time, dd_till_next_review):
        return safety_stock + dd_over_lead_time + dd_till_next_review

    reorder_point = reorder_point_calculation(safety_stock, dd_over_lead_time, dd_till_next_review)

    # Function to calculate total cost
    def calculate_total_cost(inventory_levels, order_placed, holding_cost, ordering_cost):
        average_inventory = np.mean(inventory_levels)
        total_orders = np.sum(order_placed > 0)
        total_holding_cost = average_inventory * holding_cost
        total_ordering_cost = total_orders * ordering_cost
        return total_holding_cost + total_ordering_cost

    # Monte Carlo Simulation 
    def monte_carlo_optimal_Q_cost(forecasted_demand_mean, forecasted_demand_std, lead_time, MOQ, max_shelf_space, 
                                   current_inventory, back_orders, open_orders, holding_cost, ordering_cost, 
                                   review_period=7, num_simulations=1000):
        
        total_costs = []
        optimal_Qs = []

        for _ in range(num_simulations):
            daily_demand = np.random.normal(forecasted_demand_mean, forecasted_demand_std, review_period + lead_time)
            total_forecasted_demand = np.sum(daily_demand)

            # Adjust required_qty calculation to consider back orders and open orders
            required_qty = max_shelf_space_for_SKU + back_orders - open_orders - current_inventory
            if required_qty < 0:
                required_qty = 0
            if required_qty % MOQ > MOQ // 2:
                optimal_Q = ((required_qty // MOQ) + 1) * MOQ  # Round up
            else:
                optimal_Q = (required_qty // MOQ) * MOQ  # Round down
            if optimal_Q > max_shelf_space:
                optimal_Q = max_shelf_space

            inventory_levels = [current_inventory + optimal_Q - total_forecasted_demand]
            order_placed = optimal_Q
            total_cost = calculate_total_cost(inventory_levels, order_placed, holding_cost, ordering_cost)
            total_costs.append(total_cost)
            optimal_Qs.append(optimal_Q)

        min_cost_index = np.argmin(total_costs)
        optimal_Q_min_cost = optimal_Qs[min_cost_index]
        min_total_cost = total_costs[min_cost_index]

        return optimal_Q_min_cost, min_total_cost

    # Monte Carlo Simulation parameters
    forecasted_demand_mean = dd_over_lead_time / lead_time
    forecasted_demand_std = demand_std_dev

    optimal_Q_min_cost, min_total_cost = monte_carlo_optimal_Q_cost(
        forecasted_demand_mean, 
        forecasted_demand_std, 
        lead_time, 
        MOQ, 
        max_shelf_space_for_SKU, 
        initial_inventory, 
        back_orders, 
        open_orders,
        holding_cost,
        ordering_cost
    )

    print("Optimal Order Quantity (Q) to Minimize Cost:", optimal_Q_min_cost)
    print("Minimum Total Cost:", min_total_cost)

    # Inventory System Simulation 
    try:
        todays_index = demand_data.index.get_loc(today_date)
    except KeyError:
        print(f"Date {today_date} not found in demand data index.")
        todays_index = 0  # Assuming we start from the beginning if date not found

    end_index = todays_index + simulation_days

    # Slice the demand data for the simulation period
    forecasted_demand = demand_data.iloc[todays_index:end_index]['total_qty'].values

    # Generate the days for the simulation period
    days = pd.date_range(start=today_date, periods=simulation_days)

    # Initialize the DataFrame for the simulation
    data = {
        'day_tracker': days,
        'stock_start': np.zeros(simulation_days),
        'forecasted_demand': forecasted_demand,
        'stock_end': np.zeros(simulation_days),
        'inventory_check': [False] * simulation_days,
        'order_placed': [False] * simulation_days,
        'orders_in_transit': [0] * simulation_days  # Track orders in transit
    }

    df = pd.DataFrame(data)

    # Set initial inventory, considering back orders and open orders
    df.at[0, 'stock_start'] = initial_inventory - back_orders - open_orders

    # Simulation
    for i in range(simulation_days):
        if i > 0:
            df.at[i, 'stock_start'] = df.at[i - 1, 'stock_end'] + (df.at[i - lead_time, 'orders_in_transit'] if i - lead_time >= 0 else 0)
        
        # Calculate end-of-day stock
        df.at[i, 'stock_end'] = df.at[i, 'stock_start'] - df.at[i, 'forecasted_demand']
        
        # Ensure stock_end is not negative
        if df.at[i, 'stock_end'] < 0:
            df.at[i, 'stock_end'] = 0
        
        # Inventory check and order placement
        if (i + 1) % review_period == 0:
            df.at[i, 'inventory_check'] = True
            
            if df.at[i, 'stock_end'] < reorder_point:
                df.at[i, 'order_placed'] = True
                # Place an order to restock
                df.at[i, 'orders_in_transit'] = optimal_Q_min_cost

    return df, optimal_Q_min_cost, min_total_cost

# Example usage:
if __name__ == "__main__":
    sales_order_file_path = input("Please enter the path to the sales order data file: ")
    result_df, optimal_Q, min_cost = r_Q_continuous(sales_order_file_path)
    print(result_df)
    print("Optimal Order Quantity (Q) to Minimize Cost:", optimal_Q)
    print("Minimum Total Cost:", min_cost)


Optimal Order Quantity (Q) to Minimize Cost: 114000
Minimum Total Cost: 123410.05040293386
    day_tracker  stock_start  forecasted_demand  stock_end  inventory_check  \
0    2021-04-01       9850.0                 79     9771.0            False   
1    2021-04-02       9771.0                 37     9734.0            False   
2    2021-04-03       9734.0                 42     9692.0             True   
3    2021-04-04       9692.0                 41     9651.0            False   
4    2021-04-05       9651.0                 44     9607.0            False   
..          ...          ...                ...        ...              ...   
360  2022-03-27     216543.0                 44   216499.0            False   
361  2022-03-28     216499.0                 41   216458.0            False   
362  2022-03-29     216458.0                127   216331.0             True   
363  2022-03-30     216331.0                  2   216329.0            False   
364  2022-03-31     216329.0            

/var/folders/ns/1yxzfx892f36qy2_byrlq2kw0000gn/T/ipykernel_70873/758236294.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)
